
# Получение данных через API 
  


# Зачем нужны API

`API` для аналитика это инструмент связи с данными, которые находятся внутри сторонних сайтов. Через API мы передаем команды-запросы, а взамен получаем ответ. Все API разные, но есть общие подходы

- у большинства API есть адрес, по которому нужно отправить запрос
- в документации указаны определенные требования к структуре запроса

### Почему нельзя просто парсить сайты?

Вообще-то можно, но если вы напишете робота, который прорывается через визуальный интерфейс и создает лишнюю нагрузку на сайт, то вас просто забанят.

`Визуальный интерфейс` сайтов для пользователей — он нагружен разнообразными стилями и картинками, похоже на парадный вход с красивым и внимательным консьержем, который физически не успеет обслужить роботизированный поток из тысяч запросов. 

`API` —  интерфейс для программ — как служебный вход, который снимает лишнюю нагрузку с парадного, созданного для пользователей, он не такой красивый, зато шире.

Хотя и этот служебный вход не резиновый и API прописывают `ограничения` запросов, которые можно создавать. Часто, например, владельцы API просят нас ограничить частоту обращений к сервису или объем получаемых данных.

### Открытые и закрытые API

`Открытые` — общедоступные (например, API Мирового банка). [Список публичных API](https://github.com/public-apis/public-apis)

`Закрытые` — требующие разнообразных процедур авторизации (например, API Google Drive). 

### Что можно получить от API сайтов?

Чаще всего возвращают данные в формате `json`, с которым мы и будем работать, или реже `xml`. 




## JSON

`JSON` — текстовый формат данных из `JavaScript`, но многие среды программирования имеют возможность читать и генерировать JSON, в том числе Python.

Работая с модулем `requests`, мы будем использовать метод `json()`, который преобразует ответ в объект класса `dict`. 

Подробнее: https://python-scripts.com/json

___

## Открытые API

### Задача 1 

Получаем время восхода и заката по координатам

#### Как это сделать?

1. Изучить документацию API сайта `https://sunrise-sunset.org/`
2. Получить координаты города в нужном формате
3. Отправить запрос сайту `https://sunrise-sunset.org/`, на котором есть нужные нам данные
4. Получить ответ 

__Идем на сайт, ищем страницу с документацией `API` [`https://sunrise-sunset.org/`](https://sunrise-sunset.org/) и находим ответы на главные вопросы__

1. Какие параметры, то есть данные, мы можем получить через этот API
2. В каком формате этот API принимает запрос
3. В каком формате возвращается ответ

__Мы поняли, что на вход нам понадобятся:__

1. две координаты в формате `float`
2. дата, данные за которую мы хотим получить

### 1. Получим координаты
Библиотека `geocoder` поможет нам получить координаты по географическому названию.

In [ ]:
#!pip install geocoder

import sys
!{sys.executable} -m pip install geocoder 


In [ ]:
import geocoder

Из нее возьмем функцию `.arcgis()`, которая возвращает всю информацию о переданном географическом названии.

In [ ]:
city = geocoder.arcgis('Иркутск')
print(city.json)

Широта — это `lat`, долгота — `lng`.

Создадим список, в который положим наши широту и долготу. Но вообще, мы можем сохранить их в любом удобном формате.

In [ ]:
crdnt = [city.json['lat'], city.json['lng']]
print(crdnt)

In [ ]:
crdnt=[52.2978, 104.296] # данные из другого источника

### 2. Создадим запрос 

`requests` — библиотека инструментов для обмена информацией по интернету. С ее помощью можно легко и быстро подключаться к сайтам, забирать оттуда информацию, отправлять туда какую-то информацию (например, логин и пароль для авторизации). В работе аналитика эта библиотека чаще всего используется для получения данных из сети.

Для этого используется функция `get()`, которая получает на вход URL API и параметры запроса в формате словаря

Снова идем в документацию: https://sunrise-sunset.org/api, чтобы 

1. вспомнить, через какую ссылку мы обращаемся к нему
2. вспомнить имена параметров

In [ ]:
import requests # входит в стандартную библиотеку pyton

In [ ]:
URL = 'https://api.sunrise-sunset.org/json'
params = {
    'lat': crdnt[0], # берем первый элемент из списка
    'lng': crdnt[1], # берем второй элемент из списка
    'date': '2025-02-14' # дата в формате строки (работа с API - формирование строк!)
}

sun_r_s = requests.get(URL, params=params) # requests за нас формирует конечный URL из параметров

sun_r_s

`<Response [200]>` - означает, что все верно. 

Чтобы посмотреть, что внутри ответа, используем метод `.content` 

In [ ]:
sun_r_s.content

Мы получили строку в формате `JSON`. Ее можно преобразовать в словарь методом `.json` и после этого обращаться к различным полям.

In [ ]:
sun_r_s_dict = sun_r_s.json()
sun_r_s_dict

In [ ]:
type(sun_r_s_dict)

Например, получим продолжительность дня, обратившись к значению внутреннего словаря по ключу `'day_length'`

In [ ]:
sun_r_s_dict['results']['day_length']

Если нужно, преобразуем ответ в датафрейм, передав методу `pd.DataFrame` внутренний словарь

In [ ]:
import pandas as pd

pd.DataFrame([sun_r_s.json()['results']])

___

## API World Bank

Рассмотрим работу с `API Мирового Банка`. Там мы найдем гигабайты данных, описывающих разные экономические и социальные показатели сотен стран. 

Это тоже открытый API. Перейдем на сайт [документации](https://datahelpdesk.worldbank.org/knowledgebase/topics/125589) к нему. Документация гораздо больше и сложнее. Разберемся, как она устроена.

### Best Practices

Тут перечислены разделы с информацией о том, как следует пользоваться API. Первым делом идем в раздел [Development Best Practices](https://datahelpdesk.worldbank.org/knowledgebase/articles/902064-development-best-practices). Тут собраны рекомендации по использованию API и ограничения. Хороший тон — соблюдать их. Несоблюдение таких рекомендаций может привести к поломке сервиса и бану вашего IP.

__Две главные рекомендации:__
1. Кэшировать информацию — сохранять результаты предыдущих запросов, чтобы не ходить в API без необходимости несколько раз.
2. Не делать повторных запросов при задержке ответа. Если при задержке ответа в связи с загрузкой сервера запрашивать сервис еще и еще, это приведет только к еще большей нагрузке на сервер.

### Описание интерфейса

Далее рассмотрим, какие ошибки может выдавать API, чтобы понимать, как их обрабатывать.

<table><tbody><tr><th class="wysiwyg-tmp-selected-cell">Error Code</th><th>Response Code</th><th>Description</th></tr><tr><td>105</td><td>503 'Service currently unavailable'</td><td>'The requested service is temporarily unavailable.'</td></tr><tr><td>110</td><td>404 'API Version "XXX" not found.'</td><td>'The requested API version was not found.'</td></tr><tr><td>111</td><td>404 'Format "XXX" not found.'</td><td>'The requested response format was not found.'</td></tr><tr><td>112</td><td>404 'Method "XXX" not found.'</td><td>'The requested method was not found.'</td></tr><tr><td>115</td><td>404 'Missing required parameter'</td><td>'Parameters which are required have not been sent.'</td></tr><tr><td>120</td><td>404 'Parameter "XXX" has an invalid value.'</td><td>'The provided parameter value is not valid.'</td></tr><tr><td>140</td><td>400 'Endpoint “XXX” not found.’</td><td>'The requested endpoint was not found'</td></tr><tr><td>150</td><td>400 'Language with ISO2 code: "XX" is not yet supported in the API'</td><td>'Response requested in an unsupported language.'</td></tr><tr><td>160</td><td>400 ' Filtering data-set on an indicator value without indicating a date range is meaningless and is not allowed.'</td><td>'You need to indicate date-range if you want to filter by an indicator value.'</td></tr><tr><td>199</td><td>500 'Unexpected error'</td><td>'An unexpected error was encountered while processing the request.'</td></tr></tbody></table>

При успешном запросе обычно мы получаем желанный код 200.

### Доступ к списку показателей (индикаторов)

Чтобы узнать какие показатели мы можем запросить, перейдем на [страницу](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries) с описанием соответствующего интерфейса.

Чтобы получить данные обо всех индикаторах, мы должны запросить API по такому адресу:

http://api.worldbank.org/v2/indicator

Чтобы получить данные в формате JSON, надо добавить `?format=json`

Скопируем в адресную строку браузера и посмотрим, что будет:

http://api.worldbank.org/v2/indicator?format=json

Мы получили огромный массив какого-то текста. Посмотрим, как он будет выглядеть в виде таблиц. Документация обещает такие данные:

* код
* название
* единицы измерения
* ID источника
* описание источника
* название организации, предоставляющей данные
* ID темы
* название темы


Часть `?format=json` — это параметры запроса. Вы можете их видеть в URL после символа `?`. Присмотритесь к ссылкам в своем браузере и, возможно, найдете там что-то интересное. 

`Requests` позволяет указывать параметры в виде словаря, а не приклеивать их к URL в виде текста. 

Для доступа к API снова воспользуемся функцией `get()` из  библиотеки `requests` и передадим ей два параметра:

1. `url = http://api.worldbank.org/v2/indicator`
2. `params = {'format':'json'}`

___

In [ ]:
import requests
import pandas as pd

In [ ]:
INDICATORS_URL = "http://api.worldbank.org/v2/indicator"

In [ ]:
indicators = requests.get(url=INDICATORS_URL # ссылка на API
                          , params={'format':'json'} # параметры запроса
                         )

Выведем результат.

In [ ]:
indicators

Мы не видим огромного массива текста как в браузере. Вместо этого мы видим код ответа API. И к нашему удовлетворению он равен 200, что значит, запрос получен и успешно обработан. 

А содержимое ответа хранится в `indicators.content`

In [ ]:
indicators.content

С помощью `библиотеки json` мы можем преобразовать данные в более привычный формат. Для этого импортируем библиотеку и вызовем оттуда метод  `.loads()`, который отдаст список с вложенными словарями.

In [ ]:
import json

In [ ]:
indicators_data = json.loads(indicators.content)
indicators_data

Мы получили одну страницу с 50 показателями. 
Документация обещает, что ответ будет в виде списка, где первый элемент описывает данные, а второй содержит их:
    
```json
[{
  "page": 1,
  "pages": 1,
  "per_page": "50",
  "total": 1
  },
  [{
    "id": "NY.GDP.MKTP.CD","name":
    "GDP (current US$)",
    "unit": "",
    "source": {
      "id": "2",
      "value": "World Development Indicators"},
    "sourceNote": "GDP at purchaser's prices ... ",
    "sourceOrganization": "World Bank national accounts data, and OECD National Accounts data files.",
    "topics": [
      {"id": "19","value": "Climate Change"},
      {"id": "3","value": "Economy & Growth"}
    ]
  }]
]
```
    
Проверим это и посмотрим, что у нас лежит в `indicators_data`

In [ ]:
type(indicators_data) 

Отлично, мы поняли, что внутри лежит список, давайте посмотрим его длину

In [ ]:
len(indicators_data)

В списке всего 2 элемента, посмотрим на них по очереди. 

In [ ]:
indicators_data[0] # в первом элементе списка — метаданные.

In [ ]:
indicators_data[1] # во втором — сами данные, это страндартная практика.

In [ ]:
len(indicators_data[1])

В описании данных мы видим, что всего показателей 24604, попробуем получить все показатели.

In [ ]:
indicators = requests.get(url=INDICATORS_URL # ссылка на API
                          , params={'format':'json','per_page':24604} # параметры запроса
                         )
# вместо числа 20083 можно передать ссылку на это же значение, например вот так:  indicators_data[0]['total'] 

Всё получилось!

In [ ]:
indicators

По правилам хорошего тона закрываем соединение с сервером методом `.close()`, и дальше будем работать с полученными данными

In [ ]:
indicators.connection.close()

Чтобы было удобнее изучать данные, можно записать их в DataFrame

In [ ]:
indicators_data = json.loads(indicators.content) # приводим к списку с вложенными словарями
indicators_data[0] # первый элемент — словарь с описанием данных

Как и в прошлый раз, используем `pd.DataFrame`

In [ ]:
indicators_df = pd.DataFrame(indicators_data[1]) # берем второй элемент — словарь с данными

In [ ]:
indicators_df.head()

In [ ]:
indicators_df.info()

### Задача 2 

Посмотрим динамику ВВП по странам и группам стран


Мы выгрузили все показатели, которые нам отдал World Bank, а нам нужны только ВВП стран за определенный период.

Нужно передать в запрос идентификатор показателя, который мы хотим. Идентификатор ВВП (GDP) `"NY.GDP.MKTP.KD.ZG"`

Его можно подсмотреть, изучив документацию, исследовав данные или просто загуглив :)

### Доступ к данным

Идем на страницу с описанием типичного способа обращения к API [**API Basic Call Structures**](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures)

Пример показывает, как получить данные за период времени:

http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2000:2001

1. Сначала указываем показатель в URL
2. Потом в качестве параметра передаем период

In [ ]:
URL_DATA = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.KD.ZG"

params={'format':'json',
        'date':'2000:2025'}

In [ ]:
gdp_growth = requests.get(url=URL_DATA, # строка с запросом к API
                          params=params # параметры запроса
                         )

Посмотрим на ответ:

In [ ]:
gdp_growth

In [ ]:
gdp_growth.connection.close() # закроем коннект
gdp_growth_data = json.loads(gdp_growth.content) # распарсим содержимое ответа
gdp_growth_data[0]

In [ ]:
len(gdp_growth_data[1])

Снова у нас только 1 страница, с 50 записями или строками. Давайте вытащим все записи.

In [ ]:
gdp_growth = requests.get(url=URL_DATA # ссылка на API
                          , params={'format':'json',
                                    'date':'2000:2025',
                                    'per_page':gdp_growth_data[0]['total']
                                   } # параметры запроса
                         )

In [ ]:
gdp_growth.connection.close()

Что мы получили на этот раз?

In [ ]:
### gdp_growth_data = json.loads(gdp_growth.content)
gdp_growth_data[0]

Все данные на месте, запишем их в DataFrame

In [ ]:
gdp_growth_df = pd.DataFrame(gdp_growth_data[1])

In [ ]:
gdp_growth_df.head()

Давайте немного почистим полученные данные и отрисуем динамику изменения ВВП по экономическим макрорегионам.

1. В колонке `country` лежит словарь, в котором название страны можно получить по ключу `value` . Напишем функцию, которая принимает на вход словарь и возвращает значение по ключу `value`
2. После создадим колонку `country_name`, в которую запишем результат выполнения этой функции над колонкой `country`

In [ ]:
def get_country_name(x):
    return x['value']

gdp_growth_df['country_name'] = gdp_growth_df['country'].apply(get_country_name)

Когда мы работаем с очень простыми функциями, можно не определять их, а использовать `lambda функции`. 

<img style='float:left' src="https://res.cloudinary.com/djcwxgbfz/image/upload/v1610963213/skills/Shpargalka_8.png" width="700"/> 

In [ ]:
gdp_growth_df['country_name'] = gdp_growth_df['country'].apply(lambda x: x['value']) 

Найдем `значения по годам и странам`, для этого создадим сводную таблицу с помощью метода `.pivot()`

In [ ]:
grp_growth_by_country = gdp_growth_df.pivot(index='date', columns='country_name', values='value')
grp_growth_by_country

Посмотрим, для каких стран и объединений у нас есть данные

In [ ]:
grp_growth_by_country.columns.tolist()

Рассмотрим рост ВВП для групп стран по признаку географии (как [здесь](https://datatopics.worldbank.org/world-development-indicators/the-world-by-income-and-region.html))

1. импортируем библиотеки для визуализации
2. создадим словарь с нужными группами стран
3. построим линейный график

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
country_group = ['North America'
                , 'South Asia'
                , 'Middle East & North Africa'
                , 'Europe & Central Asia'
                , 'Latin America & Caribbean'
                , 'East Asia & Pacific']

In [ ]:
plt.figure(figsize=(15,7))
sns.lineplot(data=grp_growth_by_country[country_group], linewidth=3)
plt.xlabel("Год")
plt.ylabel("Прирост ВВП год к году, %");

 <center><img src="https://cdn.skyeng.ru/resources/image/skills/python/lesson1-1/3.png" /> 